In [1]:
# Module 3 - Libraries
import time 
import sys 
import pubnub
import asyncio
import time
from kasa import SmartPlug
from pubnub.pubnub import PubNub, SubscribeListener, SubscribeCallback,PNStatusCategory 
from pubnub.pnconfiguration import PNConfiguration 
from pubnub.exceptions import PubNubException 
import warnings
warnings.filterwarnings('ignore')

In [2]:
import asyncio
import time
from kasa import SmartPlug

# Module 3 Execution
def main():
    pnconf = PNConfiguration()   
    pnconf.publish_key = 'pub-c-47de1ea1-b0ba-462c-ba6a-1c4fa5371251'
    pnconf.subscribe_key = 'sub-c-d5e026e5-08a6-401d-9651-a1ba89559278'
    pnconf.uuid = 'Project'
    channel = 'Project'
    pubnub = PubNub(pnconf)
    pubnub.add_listener(VFDSubscribeCallback())
    pubnub.subscribe().channels(channel).execute()

class light:
    msg = ""
    
class VFDSubscribeCallback(SubscribeCallback):
    def message(self, pubnub, event):
        print(event.message)
        remoteCommand = event.message["command"]
        channel = 'Project'

        if remoteCommand == "Allow": 
            light.msg = "on"
            pubnub.publish().channel(channel).message("Opening the Door!!!").sync()
            time.sleep(5)
            pubnub.publish().channel(channel).message("Automatic Closing Door Activated").sync()

        elif remoteCommand == "Deny":
            light.msg = "off"
            pubnub.publish().channel(channel).message("Closing the Door").sync()
                
async def call():
    p = SmartPlug("192.168.0.242")
    
    if (light.msg == "on"):
        await p.turn_on()
        time.sleep(3)
        await p.turn_off()
    
    if (light.msg == "off"):
        await p.turn_off()

if __name__ == '__main__':
    main()
    time.sleep(5)
    await call()